## Описание проекта
Интернет-магазин «В один клик» продаёт разные товары: для детей, для дома, мелкую бытовую технику, косметику и даже продукты. Отчёт магазина за прошлый период показал, что активность покупателей начала снижаться. Привлекать новых клиентов уже не так эффективно: о магазине и так знает большая часть целевой аудитории. Возможный выход — удерживать активность постоянных клиентов. Сделать это можно с помощью персонализированных предложений.

## Импорт библиотек

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings
import shap

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from phik.report import plot_correlation_matrix
from phik import phik_matrix

warnings.filterwarnings("ignore", category=FutureWarning)

# загружаем нужные модели
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

# загружаем функцию для работы с метриками
from sklearn.metrics import roc_auc_score

from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

## Константы

In [ ]:
RANDOM_STATE = 42
TEST_SIZE = 0.25

## Шаг 1. Загрузка данных

- *1.1 Загрузите данные.*
- *1.2 Проверьте, что данные в таблицах соответствуют описанию. Исследованием и объединением данных вы займётесь позже.*

In [ ]:
market_file = pd.read_csv('/kaggle/input/market/market_file.csv')
market_money = pd.read_csv('/kaggle/input/market/market_money.csv')
market_time = pd.read_csv('/kaggle/input/market/market_time.csv')
money = pd.read_csv('/kaggle/input/market/money.csv', sep=";", decimal=",")

In [ ]:
market_file.head()

In [ ]:
market_file.info()

In [ ]:
market_money.head()

In [ ]:
market_money.info()

In [ ]:
market_time.head()

In [ ]:
market_time.info()

In [ ]:
money.head()

In [ ]:
money.info()

**Вывод**
* Данные соответствуют описанию
* В данных отсутствуют пропуски

## Шаг 2. Предобработка данных

*Проведите необходимую предобработку данных. Названия столбцов можно оставить кириллическими.*

Определим функцию для удобства анализа

In [ ]:
def extended_dataframe_analysis(df):
    """
    Предоставляет расширенную первичную информацию по таблице данных.

    :param df: DataFrame для анализа.
    :return: Словарь с ключевой информацией.
    """
    analysis = {}

    # Пропущенные значения
    missing_values = df.isnull().sum()
    missing_percent = (missing_values / len(df)) * 100
    analysis['missing_values'] = pd.DataFrame({'count': missing_values, 'percentage': missing_percent})

    # Количество дубликатов строк
    analysis['duplicates'] = df.duplicated().sum()

    # Информация по категориальным столбцам
    categorical_cols = df.select_dtypes(include=['object', 'category']).columns
    categories_info = {col: {'unique_count': df[col].nunique(), 'unique_values': df[col].unique()} for col in categorical_cols}
    analysis['categories_info'] = categories_info

    return analysis

In [ ]:
def fancy_info_output(info):
    """
    Показывает в консоли информацию о категориальных значениях по каждому параметру

    :param info: словарь
    """
    for category in info:
        print(category, '=>', info[category]['unique_count'])
        for value in info[category]['unique_values']:
            print('-', value)

In [ ]:
market_file_analysis = extended_dataframe_analysis(market_file)
market_money_analysis = extended_dataframe_analysis(market_money)
market_time_analysis = extended_dataframe_analysis(market_time)
money_analysis = extended_dataframe_analysis(money)

In [ ]:
market_file.shape

In [ ]:
market_file.dtypes

In [ ]:
market_file_analysis['missing_values']

In [ ]:
fancy_info_output(market_file_analysis['categories_info'])

In [ ]:
market_file_analysis['duplicates']

После изучения данных из `market_file`, сделаем следующее:
* Имена параметров в shake_case для консистентности
* В значении параметра `Тип сервиса` исправим опечатку

In [ ]:
market_file = market_file.rename(columns={
    'Покупательская активность': 'Покупательская_активность',
    'Тип сервиса': 'Тип_сервиса',
    'Разрешить сообщать': 'Разрешить_сообщать'
})

In [ ]:
market_file.loc[market_file['Тип_сервиса'] == 'стандартт', 'Тип_сервиса'] = 'стандарт'

In [ ]:
market_money.shape

In [ ]:
market_money.dtypes

In [ ]:
market_money_analysis['missing_values']

In [ ]:
fancy_info_output(market_money_analysis['categories_info'])

In [ ]:
market_money_analysis['duplicates']

После изучения данных из `market_money` видим что в значении параметра `Период` есть опечатка. Исправим её.

In [ ]:
market_money.loc[market_money['Период'] == 'препредыдущий_месяц', 'Период'] = 'предыдущий_месяц'

In [ ]:
market_time.shape

In [ ]:
market_time.dtypes

In [ ]:
market_time_analysis['missing_values']

In [ ]:
fancy_info_output(market_time_analysis['categories_info'])

In [ ]:
market_time_analysis['duplicates']

После изучения данных из `market_time` видим что в значении параметра Период есть опечатка. Исправим её.

In [ ]:
market_time.loc[market_time['Период'] == 'предыдцщий_месяц', 'Период'] = 'предыдущий_месяц'

In [ ]:
money.shape

In [ ]:
money.dtypes

In [ ]:
money_analysis['missing_values']

In [ ]:
money_analysis['duplicates']

**Вывод:**
* пропущенных значений не обнаружено
* дубликатов тоже в данных нет
* типы все верные
* приведены к общему виду имена параметров
* исправлены опечатки

## Шаг 3. Исследовательский анализ данных

*Проведите исследовательский анализ данных из каждой таблицы. Отберите клиентов с покупательской активностью не менее трёх месяцев, то есть таких, которые что-либо покупали в этот период. Оформите выводы по результатам шага.*

На данном шаге
- изучим статистики всех таблиц и выбросы с помощью диаграммы "ящик с усами".
- отберем клиентов с покупательской активностью не менее трёх месяцев



Определим функции для построения диаграмм

In [ ]:
def plot_histograms_boxplots(df):
    """
    Строит гистограммы и ящики с усами для каждого количественного фактора в DataFrame.

    :param df: DataFrame, содержащий данные
    """
    # Выбор числовых столбцов
    numeric_cols = df.select_dtypes(include=['number']).columns

    # Установка размера фигуры
    num_plots = len(numeric_cols)
    plt.figure(figsize=(10, 5 * num_plots))

    # Построение гистограмм и ящиков с усами для каждого числового столбца
    for i, col in enumerate(numeric_cols):
        # Гистограмма
        plt.subplot(num_plots, 2, 2*i + 1)
        df[col].hist(bins=15)
        plt.title(f'Histogram of {col}')
        plt.xlabel(col)
        plt.ylabel('Frequency')

        # Ящик с усами
        plt.subplot(num_plots, 2, 2*i + 2)
        df.boxplot(column=col)
        plt.title(f'Box Plot of {col}')

    plt.tight_layout()
    plt.show()

In [ ]:
def plot_countplots(df):
    """
    Строит countplots для каждого категориального фактора в DataFrame.

    :param df: DataFrame, содержащий данные.
    """
    # Выбор категориальных столбцов
    categorical_cols = df.select_dtypes(exclude=['number']).columns

    # Установка размера фигуры
    num_plots = len(categorical_cols)
    plt.figure(figsize=(10, 5 * num_plots))

    # Построение countplots для каждого категориального столбца
    for i, col in enumerate(categorical_cols):
        plt.subplot(num_plots, 1, i + 1)
        sns.countplot(y=col, data=df)
        plt.title(f'График {col}')
        plt.xlabel(col)
        plt.ylabel('Количество')

    plt.tight_layout()
    plt.show()

Чтобы id не учитывались в анализе сделаем его индексом

In [ ]:
market_file = market_file.set_index('id')
market_money = market_money.set_index('id')
market_time = market_time.set_index('id')
money = money.set_index('id')

In [ ]:
market_file.describe()

In [ ]:
plot_histograms_boxplots(market_file)

Диаграмма для параметра `Маркет_актив_тек_мес` выглядит так потому что уникальных значений всего 3

In [ ]:
market_file['Маркет_актив_тек_мес'].unique()

В параметрах `Маркет_актив_6_мес`, `Акционные_покупки`, `Неоплаченные_продукты_штук_квартал` присутствуют выбросы

In [ ]:
# Изучим распределение
plot_countplots(market_file)

В параметрах: `Тип_сервиса` и `Разрешить_сообщать` виден дисбаланс в значениях. Тажке он есть и в целевом признаке `Покупательская_активность`

In [ ]:
market_money.describe()

In [ ]:
plot_histograms_boxplots(market_money)

Диаграмму для параметра `Выручка` так "сплющело" из-за выброса, найдем и удалим его

In [ ]:
market_money.sort_values(by='Выручка', ascending=False).head()

In [ ]:
market_money = market_money[market_money['Выручка'] != 106862.2]
plot_histograms_boxplots(market_money)

Теперь значение 0, похоже на выброс, тоже удаляем

In [ ]:
market_money = market_money[market_money['Выручка'] != 0]
plot_histograms_boxplots(market_money)

In [ ]:
# Изучим распределение
plot_countplots(market_money)

In [ ]:
market_time.describe()

In [ ]:
plot_histograms_boxplots(market_time)

Тут всё гуд

In [ ]:
# Изучим распределение
plot_countplots(market_time)

Такое бывает?!

In [ ]:
money.describe()

In [ ]:
plot_histograms_boxplots(money)

Параметр `Прибыль` имеет выбросы

Чтобы отобрать пользователей с покупательской активностью не менее трёх месяцев можно просто взять пользователей из таблицы money. Данныя таблица как раз хранит данные о среднемесячной прибыли покупателя за последние 3 месяца. Так как в таблице нет значений равные 0 следовательно все пользователи совершали покупки в течении нужного срока

**Вывод**

В данных присутствуют выбросы и наблюдается неравномерное распределение в данных.

## Шаг 4. Объединение таблиц

- *4.1 Объедините таблицы market_file.csv, market_money.csv, market_time.csv. Данные о прибыли из файла money.csv при моделировании вам не понадобятся.*
- *4.2 Учитывайте, что данные о выручке и времени на сайте находятся в одном столбце для всех периодов. В итоговой таблице сделайте отдельный столбец для каждого периода.*

Перед тем как объядинять данные явно переименуем столблец `Период` в таблице `market_money` на `Период_деньги`, а `Период` в таблице `market_time` на `Период_время`

In [ ]:
market_money.columns

In [ ]:
market_money = market_money.rename(columns={'Период': 'Период_деньги'})
market_money.columns

In [ ]:
market_time.columns

In [ ]:
market_time = market_time.rename(columns={'Период': 'Период_время'})
market_time.columns

In [ ]:
df_full = market_file.join(market_money, on='id')
df_full = df_full.join(market_time, on='id')

In [ ]:
df_full.head(20)

In [ ]:
df_full.shape

In [ ]:
df_full.info()

**Вывод:** Кажется объединение прошло успешно. Типы в норме и нет NaN

## Шаг 5. Корреляционный анализ

*Проведите корреляционный анализ признаков в количественной шкале в итоговой таблице для моделирования. Сделайте выводы о мультиколлинеарности и при необходимости устраните её.*

In [ ]:
sns.pairplot(df_full, hue='Покупательская_активность')

plt.show()

In [ ]:
numeric_cols = df_full.select_dtypes(include=['number']).columns
correlation_matrix = df_full[numeric_cols].corr(method='spearman')

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.show()

## Шаг 6. Использование пайплайнов

*Примените все изученные модели. Для этого используйте пайплайны.*

- *6.1 Во время подготовки данных используйте ColumnTransformer. Количественные и категориальные признаки обработайте в пайплайне раздельно. Для кодирования категориальных признаков используйте как минимум два кодировщика, для масштабирования количественных — как минимум два скейлера.*

*Напоминаем, что для каждой модели можно подготовить данные с разным кодированием и масштабированием.*

- *6.2 Обучите четыре модели: KNeighborsClassifier(), DecisionTreeClassifier(), LogisticRegression() и  SVC(). Для каждой из них подберите как минимум один гиперпараметр. Выберите подходящую для задачи метрику, аргументируйте свой выбор. Используйте эту метрику при подборе гиперпараметров.*
- *6.3 Выберите лучшую модель, используя заданную метрику. Для этого примените одну из стратегий:*
    - *использовать пайплайны и инструменты подбора гиперпараметров для каждой модели отдельно, чтобы выбрать лучшую модель самостоятельно*
    - *использовать один общий пайплайн для всех моделей и инструмент подбора гиперпараметров, который вернёт вам лучшую модель*

In [ ]:
encoder = LabelEncoder()

X = df_full.drop(['Покупательская_активность'], axis=1)
y = encoder.fit_transform(df_full['Покупательская_активность'])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size = TEST_SIZE,
    random_state = RANDOM_STATE,
    stratify = y)

In [ ]:
ohe_columns = ['Тип_сервиса', 'Разрешить_сообщать', 'Период_деньги', 'Период_время']
ord_columns = ['Популярная_категория']
num_columns = ['Маркет_актив_6_мес', 'Маркет_актив_тек_мес', 'Длительность',
               'Акционные_покупки', 'Средний_просмотр_категорий_за_визит',
               'Неоплаченные_продукты_штук_квартал', 'Ошибка_сервиса', 'Выручка',
               'Страниц_за_визит', 'минут']

In [ ]:
# создаём общий пайплайн для подготовки данных
data_preprocessor = ColumnTransformer([
    ('ohe', OneHotEncoder(drop='first', handle_unknown='error'), ohe_columns),
    ('ord', OrdinalEncoder(), ord_columns),
    ('num', StandardScaler(), num_columns)
], remainder='passthrough')

# создаём итоговый пайплайн: подготовка данных и модель
pipe_final = Pipeline([
    ('preprocessor', data_preprocessor),
    ('models', DecisionTreeClassifier(random_state=RANDOM_STATE))
])

param_grid = [
    # словарь для модели DecisionTreeClassifier()
    {
        'models': [DecisionTreeClassifier(random_state=RANDOM_STATE)],
        'models__max_depth': range(2, 5),
        'models__max_features': range(2, 5)
    },
    # словарь для модели KNeighborsClassifier()
    {
        'models': [KNeighborsClassifier()],
        'models__n_neighbors': range(5, 100)
    },

    # словарь для модели LogisticRegression()
    {
        'models': [LogisticRegression(random_state=RANDOM_STATE)],
        'models__C': [0.1, 1.0, 10.0, 100.0]
    },
    # словарь для модели SVC()
    {
        'models': [SVC(probability=True)],
        'models__kernel': ['linear', 'rbf']
    }
]

randomized_search = RandomizedSearchCV(
    pipe_final,
    param_grid,
    cv=5,
    scoring='roc_auc',
    random_state=RANDOM_STATE,
    n_jobs=-1
)
randomized_search.fit(X_train, y_train)

print('Лучшая модель и её параметры:\n\n', randomized_search.best_estimator_)
print ('Метрика лучшей модели на тренировочной выборке:', randomized_search.best_score_)

# проверьте работу модели на тестовой выборке
# рассчитайте прогноз на тестовых данных
y_test_pred = randomized_search.predict_proba(X_test)
print(f'Метрика ROC-AUC на тестовой выборке: {roc_auc_score(y_test, y_test_pred[:, 1])}')

**Вывод:** Модель чудовищна хороша

## Шаг 7. Анализ важности признаков

- *7.1 Оцените важность признаков для лучшей модели и постройте график важности с помощью метода SHAP.*
- *7.2 Сделайте выводы о значимости признаков:*
    - *какие признаки мало значимы для модели*
    - *какие признаки сильнее всего влияют на целевой признак*
    - *как можно использовать эти наблюдения при моделировании и принятии бизнес-решений*

In [ ]:
COUNT = 10

# Извлечение лучшей модели из результатов RandomizedSearchCV
best_model = randomized_search.best_estimator_.named_steps['models']

# Предобработка данных через пайплайн без конечной модели
preprocessor = randomized_search.best_estimator_.named_steps['preprocessor']
X_train_preprocessed = preprocessor.transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)

# Получаем имена признаков после OneHotEncoder
ohe_feature_names = preprocessor.named_transformers_['ohe'].get_feature_names_out(input_features=ohe_columns)

# Для OrdinalEncoder и StandardScaler мы можем использовать имена как есть
ord_feature_names = ord_columns
num_feature_names = num_columns

# Объединяем все имена признаков в один список
all_feature_names = np.concatenate([ohe_feature_names, ord_feature_names, num_feature_names])

# Создаем DataFrame с соответствующими именами колонок
X_train_preprocessed_df = pd.DataFrame(X_train_preprocessed, columns=all_feature_names)
X_test_preprocessed_df = pd.DataFrame(X_test_preprocessed, columns=all_feature_names)

# Семпл данных для KernelExplainer
X_train_preprocessed_smpl = shap.sample(X_train_preprocessed_df, COUNT, random_state=RANDOM_STATE)
X_test_preprocessed_smpl = shap.sample(X_test_preprocessed_df, COUNT, random_state=RANDOM_STATE)

# Теперь, когда у нас есть DataFrame с именами признаков, мы можем использовать KernelExplainer
explainer = shap.KernelExplainer(best_model.predict_proba, X_train_preprocessed_smpl)
shap_values = explainer.shap_values(X_test_preprocessed_smpl)

# Построение графика с именами признаков
shap.summary_plot(shap_values, X_test_preprocessed_smpl)

## Шаг 8. Сегментация покупателей

- *8.1 Выполните сегментацию покупателей. Используйте результаты моделирования и данные о прибыльности покупателей.*
- *8.2 Выберите группу покупателей и предложите, как увеличить её покупательскую активность:*
    - *Проведите графическое и аналитическое исследование группы покупателей*
    - *Сделайте предложения по работе с сегментом для увеличения покупательской активности*
- *8.3 Сделайте выводы о сегментах:*
    - *какой сегмент вы взяли для дополнительного исследования*
    - *какие предложения вы сделали и почему*

In [ ]:
# Чтобы наверника
threshold = 0.9

# 8.1 Выполните сегментацию покупателей. Используйте результаты моделирования и данные о прибыльности покупателей.
# Применим модель для всех данных
best_model = randomized_search.best_estimator_
predictions = best_model.predict_proba(X)[:, 1]
prediction_flags = np.where(predictions > threshold, True, False)

In [ ]:
# Выберем только те данные для которых модель предсказала снижение
df_decline = df_full.loc[prediction_flags]

# Добавить в эти данные инфу о прибыльности из таблицы money
df_decline = df_decline.join(money, on='id')

# Проверим что появился столбец
df_decline.info()

In [ ]:
# 8.2 Выберите группу покупателей и предложите, как увеличить её покупательскую активность:
#     - Проведите графическое и аналитическое исследование группы покупателей
#     - Сделайте предложения по работе с сегментом для увеличения покупательской активности
# У нас есть уже список параметров которые влияют на снижение покупательской активности.
# Возьмем первые 5 и выделим группу на основе них
# Акционные_покупки, минут, Страниц_за_визит, Средний_просмотр_категорий_за_визит, Популярная_категория

plt.figure(figsize=(21, 7))

plt.subplot(1, 3, 1)
sns.histplot(df_decline['Акционные_покупки'], bins=10)
plt.title('Акционные покупки')

plt.subplot(1, 3, 2)
sns.histplot(df_decline['минут'], bins=10)
plt.title('Минут')

plt.subplot(1, 3, 3)
sns.histplot(df_decline['Страниц_за_визит'], bins=20)
plt.title('Страница за визит')

plt.tight_layout()
plt.show()

plt.figure(figsize=(20, 10))

plt.subplot(1, 2, 1)
sns.histplot(df_decline['Средний_просмотр_категорий_за_визит'], bins=10)
plt.title('Средний просмотр категорий за визит')

plt.subplot(1, 2, 2)
sns.histplot(df_decline['Популярная_категория'], bins=10)
plt.title('Популярная категория')
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

Я взял сегмент покупателей с параметрами которые сильнее всего влияют на снижение покупательской активности.

На графиках видны следующие закономености:
- Время проведенные на сайте, стрниц за визит и просмотр категорий ниже нормального распередения.
- Виден выброс на графике с акционными товарами

## Шаг 9. Общий вывод

В начале проекта были поставлены следующие задачи
- 1 - Построить модель для предсказания снижения покупательской активноски
- 2 - Используя данные модели и данные о прибыльности клиентов, выделить сегменты покупателей и разработать для них персонализированные предложения.

В качестве данных были предоставлены 4 таблицы
- market_file.csv. Таблица с данными о поведении покупателя на сайте, о коммуникациях с покупателем и его продуктовом поведении.
- market_money.csv. Таблица с данными о выручке, которую получает магазин с покупателя, то есть сколько покупатель всего потратил за период взаимодействия с сайтом.
- market_time.csv. Таблица с данными о времени (в минутах), которое покупатель провёл на сайте в течение периода.
- money.csv. Таблица с данными о среднемесячной прибыли покупателя за последние 3 месяца: какую прибыль получает магазин от продаж каждому покупателю.

Для посика лучшей модели были проделаны следующие шаги
- 1 - изучены данные
- 2 - обработаны пропуски и дубликаты
- 3 - исследованы на качество, проверены на выбросы и аномалии
- 4 - исследованы взаимосвязи количественных признаков с помощью корреляционного анализа
- 4 - объединены данные чтобы у модели были больше признаков для обучения
- 5 - настроенна автоматизация обучения с помощью пайпдайнов, в которых были обучены 2 типа моделей: линейные и на основе векторов
- 6 - лучшая модель оказалась KNeighborsClassifier

На основе выбранной модели был построен график с параметрами, которые сильнее всего влияют предсказание. На основе этих параметров был сделан вывод, что при снижении покупательской активности стоит акцентировать внимание пользователей на акционные товары в самых просматриваемых категориях.